In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from scipy.special import logsumexp


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import optuna

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Isometric Log-Ratio Transformation
def ilr_transform(X):
    epsilon = 1e-10
    X = X + epsilon
    return np.log(X / np.mean(X, axis=1, keepdims=True))

In [5]:
# Centered Log-Ratio Transformation
from scipy.stats import gmean

def clr_transform(X, pseudocount=1e-6):
    X += pseudocount
    geometric_means = gmean(otu_values, axis=1, keepdims=True)
    clr_values = np.log(otu_values / geometric_means)
    
    return clr_values

In [7]:
t2d = pd.read_csv("../qiime2/relative-frequency/genus_transposed.tsv", sep="\t")
t2d = t2d.rename(columns={"Unnamed: 0":"sample-id"})

In [9]:
t2d.head()

,sample-id,d__Bacteria;p__Bacillota_A_368345;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;__,d__Bacteria;p__Bacillota_I;c__Bacilli_A;o__Erysipelotrichales;f__Coprobacillaceae;g__Faecalibacillus,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella,d__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Burkholderiales;f__Burkholderiaceae_A_595427;g__Sutterella,d__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Enterobacterales_737866;f__Enterobacteriaceae_A_725029;g__Enterobacter_B_683926,d__Bacteria;p__Bacillota_A_368345;c__Clostridia_258483;o__Oscillospirales;f__Ruminococcaceae;g__Faecalibacterium,d__Bacteria;p__Bacillota_I;c__Bacilli_A;o__Erysipelotrichales;f__Erysipelotrichaceae;g__Holdemanella,d__Bacteria;p__Bacillota_A_368345;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Lachnospira,d__Bacteria;p__Actinomycetota;c__Actinomycetes;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium_388775,...,d__Bacteria;p__Bacillota_I;c__Bacilli_A;o__Lactobacillales;f__Lactobacillaceae;g__Fructilactobacillus,d__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Burkholderiales;f__Burkholderiaceae_A_595427;g__,d__Bacteria;p__Bacillota_A_368345;c__Clostridia_258483;o__Lachnospirales;f__Anaerotignaceae;g__Fimicola,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Muribaculaceae;g__SFTJ01,d__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Enterobacterales_737866;f__Enterobacteriaceae_A_729055;g__Serratia_L_726994,d__Bacteria;p__Bacillota_A_368345;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Shuttleworthia,d__Archaea;p__Thermoplasmatota;c__Thermoplasmata_1773;o__Methanomassiliicoccales;f__Methanomethylophilaceae;g__Methanomethylophilus,d__Bacteria;p__Bacillota_A_368345;c__Clostridia_258483;o__Oscillospirales;f__Acutalibacteraceae;g__Limousia,d__Bacteria;p__Actinomycetota;c__Actinomycetes;o__Actinomycetales;f__Cellulomonadaceae;g__Populibacterium,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__UBA932;g__Bact-08
0,SRR16113010,0.130579,0.04775,0.260685,0.010117,0.004633,0.015696,0.014561,0.000000,0.160647,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SRR16112701,0.067886,0.00000,0.000000,0.000000,0.067365,0.000069,0.000000,0.000000,0.275813,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SRR9679959,0.023116,0.00033,0.204478,0.005746,0.000000,0.038108,0.000000,0.001849,0.141866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ERR9763111,0.089366,0.00000,0.044211,0.000000,0.000111,0.121484,0.003439,0.000000,0.296777,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ERR9763104,0.062183,0.00000,0.010584,0.000000,0.000000,0.095590,0.000551,0.000221,0.031643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
sample_ids = t2d.iloc[:, 0]
otu_values = t2d.iloc[:, 1:].values

In [13]:
ilr_transformed_values = ilr_transform(otu_values)

In [15]:
clr_transformed_values = clr_transform(otu_values)

In [17]:
metadata = pd.read_csv("../qiime2/our-metadata.tsv", sep="\t")

In [19]:
metadata.head()

,sample-id,bioproject,sample-alias,t2d,country,age,bmi,sex,metformin,diet-type
0,SRR12599850,PRJNA661673,NaN,Yes,China,NaN,NaN,NaN,NaN,NaN
1,SRR12599861,PRJNA661673,NaN,Yes,China,NaN,NaN,NaN,NaN,NaN
2,SRR12599862,PRJNA661673,NaN,Yes,China,NaN,NaN,NaN,NaN,NaN
3,SRR12599863,PRJNA661673,NaN,Yes,China,NaN,NaN,NaN,NaN,NaN
4,SRR12599864,PRJNA661673,NaN,Yes,China,NaN,NaN,NaN,NaN,NaN


In [21]:
X_data = clr_transformed_values

In [23]:
Y_labels = t2d.merge(metadata, on="sample-id")['t2d'].apply(lambda x: 1 if x=='Yes' else 0).values

In [25]:
X_tensor = torch.tensor(X_data, dtype=torch.float32)
Y_tensor = torch.tensor(Y_labels, dtype=torch.float32)

In [27]:
dataset = TensorDataset(X_tensor, Y_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

## Modified VAE Architecture

In [29]:
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim_z1, latent_dim_z2):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim + 1, latent_dim_z1 + latent_dim_z2)
        self.fc2_mu_z1 = nn.Linear(latent_dim_z1 + latent_dim_z2, latent_dim_z1)
        self.fc2_logvar_z1 = nn.Linear(latent_dim_z1 + latent_dim_z2, latent_dim_z1)
        self.fc2_mu_z2 = nn.Linear(latent_dim_z1 + latent_dim_z2, latent_dim_z2)
        self.fc2_logvar_z2 = nn.Linear(latent_dim_z1 + latent_dim_z2, latent_dim_z2)

    def forward(self, x, y):
        y = y.view(-1, 1)
        x_cond = torch.cat([x, y], dim=1)  # Concatenate x and y
        h = F.relu(self.fc1(x_cond))
        
        mu_z1 = self.fc2_mu_z1(h)
        logvar_z1 = self.fc2_logvar_z1(h)
        mu_z2 = self.fc2_mu_z2(h)
        logvar_z2 = self.fc2_logvar_z2(h)

        return mu_z1, logvar_z1, mu_z2, logvar_z2

class Decoder(nn.Module):
    def __init__(self, latent_dim_z1, latent_dim_z2, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim_z1 + latent_dim_z2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z1, z2):
        z = torch.cat([z1, z2], dim=1)  # Concatenate z1 and z2
        h = F.relu(self.fc1(z))
        return torch.sigmoid(self.fc2(h))

class ModifiedVAE(nn.Module):
    def __init__(self, input_dim, latent_dim_z1, latent_dim_z2, hidden_dim):
        super(ModifiedVAE, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim_z1, latent_dim_z2)
        self.decoder = Decoder(latent_dim_z1, latent_dim_z2, hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)  # Sample from a standard normal
        return mu + eps * std  # Sample from the latent distribution

    def forward(self, x, y):
        mu_z1, logvar_z1, mu_z2, logvar_z2 = self.encoder(x, y)
        z1 = self.reparameterize(mu_z1, logvar_z1)
        z2 = self.reparameterize(mu_z2, logvar_z2)
        recon_x = self.decoder(z1, z2)
        return recon_x, mu_z1, logvar_z1, mu_z2, logvar_z2


def loss_function(recon_x, x, mu_z1, logvar_z1, mu_z2, logvar_z2, y, causal_reg=0.1):
    reconstruction_loss = F.mse_loss(recon_x, x, reduction='mean')

    # KL Divergence for z2 (unconditional prior)
    KL_z2 = -0.5 * torch.sum(1 + logvar_z2 - mu_z2.pow(2) - logvar_z2.exp())

    # Conditional prior for z1
    mu_Y = torch.where(y.unsqueeze(1) == 1, 
                       torch.full_like(mu_z1, 2.0, device=y.device), 
                       torch.zeros_like(mu_z1))
    KL_z1 = -0.5 * torch.sum(1 + logvar_z1 - ((mu_z1 - mu_Y).pow(2) + logvar_z1.exp()))

    # Use summing or apply a mechanism to handle dimension match
    # Example: using a simple method that combines information without direct multiplication
    causal_loss = causal_reg * (mu_z1.mean(dim=1) @ mu_z2.mean(dim=1))  # Dot product across means

    return reconstruction_loss + KL_z1 + KL_z2 + causal_loss

In [33]:
def train(model, data_loader, optimizer, epochs, causal_reg):
    model.train()
    total_loss = 0
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y in data_loader:
            optimizer.zero_grad()
            recon_x, mu_z1, logvar_z1, mu_z2, logvar_z2 = model(x, y)
            loss = loss_function(recon_x, x, mu_z1, logvar_z1, mu_z2, logvar_z2, y, causal_reg)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(data_loader)}")
        total_loss += epoch_loss / len(data_loader)
    
    return total_loss / epochs

In [552]:
def objective(trial):
    # Hyperparameter tuning
    latent_dim_z1 = trial.suggest_int("latent_dim_z1", 10, 80)
    latent_dim_z2 = trial.suggest_int("latent_dim_z2", 10, 80)
    hidden_dim = trial.suggest_int("hidden_dim", 10, 260)

    epochs = trial.suggest_int("epochs", 10, 200)
    causal_reg = trial.suggest_float("causal_reg", 0.001, 1.0)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)

    model = ModifiedVAE(614, latent_dim_z1, latent_dim_z2, hidden_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    avg_loss = train(model, data_loader, optimizer, epochs, causal_reg)
    
    return avg_loss

In [554]:
study = optuna.create_study(direction="minimize")  # Minimize the validation loss
study.optimize(objective, n_trials=100)


print("Best hyperparameters: ", study.best_params)
print("Best validation loss: ", study.best_value)

[I 2025-03-09 15:49:56,347] A new study created in memory with name: no-name-b506a009-454a-423e-a4f9-64b1dc67260d
/var/folders/02/jj1jlsn97sj550kl_82_8lp40000gn/T/ipykernel_11945/917332750.py:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



Epoch 1, Loss: 264190.89565629227
Epoch 2, Loss: 336.59163841834436
Epoch 3, Loss: 250.99523397592398
Epoch 4, Loss: 225.02817212618314
Epoch 5, Loss: 219.83715233435998
Epoch 6, Loss: 218.83264688345102
Epoch 7, Loss: 218.4911135160006
Epoch 8, Loss: 218.28239470261795
Epoch 9, Loss: 218.3807883629432
Epoch 10, Loss: 218.12854444063626
Epoch 11, Loss: 218.13764982957107
Epoch 12, Loss: 218.32415360670822
Epoch 13, Loss: 218.1219740647536
Epoch 14, Loss: 218.18657684326172
Epoch 15, Loss: 218.01918910099909
Epoch 16, Loss: 218.10800669743463
Epoch 17, Loss: 217.93098361675555
Epoch 18, Loss: 218.16925929142877
Epoch 19, Loss: 217.89206783588116
Epoch 20, Loss: 218.00320728008563
Epoch 21, Loss: 217.87613325852615
Epoch 22, Loss: 217.89228409987228
Epoch 23, Loss: 218.04623295710638
Epoch 24, Loss: 217.98059258094202
Epoch 25, Loss: 218.0547764117901
Epoch 26, Loss: 218.02867948091946
Epoch 27, Loss: 217.8291605435885
Epoch 28, Loss: 218.00534556462213
Epoch 29, Loss: 217.88688131479117

[I 2025-03-09 15:50:00,433] Trial 0 finished with value: 1565.7153605883288 and parameters: {'latent_dim_z1': 15, 'latent_dim_z2': 15, 'hidden_dim': 199, 'epochs': 196, 'causal_reg': 0.91338050936866, 'learning_rate': 0.023336709822344474}. Best is trial 0 with value: 1565.7153605883288.


Epoch 194, Loss: 218.8953484755296
Epoch 195, Loss: 217.8029984694261
Epoch 196, Loss: 218.10612751887396
Epoch 1, Loss: 1986.1828308105469
Epoch 2, Loss: 1430.7750537578877
Epoch 3, Loss: 1103.8180189866287
Epoch 4, Loss: 851.952883206881
Epoch 5, Loss: 726.6383555485652
Epoch 6, Loss: 655.610245337853
Epoch 7, Loss: 607.994861309345
Epoch 8, Loss: 574.2914111797626
Epoch 9, Loss: 542.7821643535907
Epoch 10, Loss: 516.2769329364484
Epoch 11, Loss: 494.4765853881836
Epoch 12, Loss: 472.2635803222656
Epoch 13, Loss: 451.9280994121845
Epoch 14, Loss: 436.6460659320538
Epoch 15, Loss: 415.4760932922363
Epoch 16, Loss: 400.4168959397536
Epoch 17, Loss: 385.9684143066406
Epoch 18, Loss: 370.92870330810547
Epoch 19, Loss: 354.39316059992865
Epoch 20, Loss: 340.1526371882512
Epoch 21, Loss: 327.86404771071216
Epoch 22, Loss: 317.1145263084999
Epoch 23, Loss: 305.15957494882434
Epoch 24, Loss: 290.08043817373425
Epoch 25, Loss: 278.1753771855281
Epoch 26, Loss: 267.83294560359076
Epoch 27, Los

[I 2025-03-09 15:50:01,266] Trial 1 finished with value: 441.94031427456787 and parameters: {'latent_dim_z1': 43, 'latent_dim_z2': 74, 'hidden_dim': 30, 'epochs': 40, 'causal_reg': 0.16875011792445635, 'learning_rate': 0.00011444816788743448}. Best is trial 1 with value: 441.94031427456787.


...

Epoch 108, Loss: 10.385980441020084
Epoch 109, Loss: 9.693172308114859
Epoch 110, Loss: 8.767926619603085
Epoch 111, Loss: 8.613183956879835
Epoch 112, Loss: 8.003444451552172
Epoch 113, Loss: 7.721076451815092
Epoch 114, Loss: 7.30936965575585
Epoch 1, Loss: 296.2862812922551
Epoch 2, Loss: 110.10136611645038
Epoch 3, Loss: 78.54887221409724
Epoch 4, Loss: 65.91952463296744
Epoch 5, Loss: 57.72602345393254
Epoch 6, Loss: 58.94891731555645
Epoch 7, Loss: 52.60964797093318
Epoch 8, Loss: 58.04936544711773
Epoch 9, Loss: 44.477341211759125
Epoch 10, Loss: 39.23617392319899
Epoch 11, Loss: 35.9539213180542
Epoch 12, Loss: 33.14836120605469
Epoch 13, Loss: 30.31531755740826
Epoch 14, Loss: 34.93520399240347
Epoch 15, Loss: 33.491503312037544
Epoch 16, Loss: 29.134415809924786
Epoch 17, Loss: 25.794834797198956
Epoch 18, Loss: 23.10548081764808
Epoch 19, Loss: 22.796964663725632
Epoch 20, Loss: 20.74856075873742
Epoch 21, Loss: 26.949381571549637
Epoch 22, Loss: 34.08703026404748
Epoch 23, 

[I 2025-03-09 15:54:30,827] Trial 98 finished with value: 18.337572366233875 and parameters: {'latent_dim_z1': 10, 'latent_dim_z2': 67, 'hidden_dim': 86, 'epochs': 127, 'causal_reg': 0.8866674602161182, 'learning_rate': 0.0037838983469805844}. Best is trial 92 with value: 16.88326444886553.


Epoch 126, Loss: 6.787474834001982
Epoch 127, Loss: 7.504777083030114
Epoch 1, Loss: 324.60590010422925
Epoch 2, Loss: 121.544705757728
Epoch 3, Loss: 85.5263855273907
Epoch 4, Loss: 69.21685438889723
Epoch 5, Loss: 62.886167819683365
Epoch 6, Loss: 57.71723226400522
Epoch 7, Loss: 51.66230077009935
Epoch 8, Loss: 46.32523419306828
Epoch 9, Loss: 42.93208617430467
Epoch 10, Loss: 40.06256734407865
Epoch 11, Loss: 37.329230528611404
Epoch 12, Loss: 37.333514580359825
Epoch 13, Loss: 34.022082768953766
Epoch 14, Loss: 38.79668485201322
Epoch 15, Loss: 31.238061097952034
Epoch 16, Loss: 28.09033489227295
Epoch 17, Loss: 27.559995284447304
Epoch 18, Loss: 27.108303400186394
Epoch 19, Loss: 28.059586084806003
Epoch 20, Loss: 23.08883391893827
Epoch 21, Loss: 20.707614715282734
Epoch 22, Loss: 22.85562016413762
Epoch 23, Loss: 20.1578647173368
Epoch 24, Loss: 18.576426909520077
Epoch 25, Loss: 18.997704615959755
Epoch 26, Loss: 18.123489453242374
Epoch 27, Loss: 17.0590348977309
Epoch 28, Lo

[I 2025-03-09 15:54:33,739] Trial 99 finished with value: 18.943947465978624 and parameters: {'latent_dim_z1': 10, 'latent_dim_z2': 67, 'hidden_dim': 87, 'epochs': 127, 'causal_reg': 0.889392475952073, 'learning_rate': 0.0040378214022114296}. Best is trial 92 with value: 16.88326444886553.


Epoch 120, Loss: 8.709141346124502
Epoch 121, Loss: 7.532064584585337
Epoch 122, Loss: 6.776101992680476
Epoch 123, Loss: 6.871937366632315
Epoch 124, Loss: 7.402101480043852
Epoch 125, Loss: 7.688502568465013
Epoch 126, Loss: 7.172717901376577
Epoch 127, Loss: 6.92516933954679
Best hyperparameters:  {'latent_dim_z1': 10, 'latent_dim_z2': 67, 'hidden_dim': 70, 'epochs': 121, 'causal_reg': 0.9325055584750703, 'learning_rate': 0.0020515766054521826}
Best validation loss:  16.88326444886553


In [555]:
print("Best hyperparameters: ", study.best_params)
print("Best validation loss: ", study.best_value)

Best hyperparameters:  {'latent_dim_z1': 10, 'latent_dim_z2': 67, 'hidden_dim': 70, 'epochs': 121, 'causal_reg': 0.9325055584750703, 'learning_rate': 0.0020515766054521826}
Best validation loss:  16.88326444886553


In [556]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)

In [37]:
#CLR attempt
best_latent_dim_z1 = 10
best_latent_dim_z2 = 67
best_hidden_dim = 70
best_epochs = 121
best_learning_rate = 0.0020515766054521826
best_causal_reg = 0.9

model_clr = ModifiedVAE(614, best_latent_dim_z1,best_latent_dim_z2,best_hidden_dim)
optimizer = torch.optim.Adam(model_clr.parameters(), lr=best_learning_rate)

train(model_clr, data_loader, optimizer, best_epochs, best_causal_reg)

Epoch 1, Loss: 342.8579707512489
Epoch 2, Loss: 129.2298360971304
Epoch 3, Loss: 88.45502031766452
Epoch 4, Loss: 73.79098180624155
Epoch 5, Loss: 65.06774359482985
Epoch 6, Loss: 56.68579145578238
Epoch 7, Loss: 46.46924899174617
Epoch 8, Loss: 44.480110535254845
Epoch 9, Loss: 39.200793999892014
Epoch 10, Loss: 34.45734251462496
Epoch 11, Loss: 32.83046472989596
Epoch 12, Loss: 31.32878982103788
Epoch 13, Loss: 29.982693378741924
Epoch 14, Loss: 27.2298552439763
Epoch 15, Loss: 25.00336764408992
Epoch 16, Loss: 21.68213811287513
Epoch 17, Loss: 20.155039053696854
Epoch 18, Loss: 21.111775948451115
Epoch 19, Loss: 19.963946855985203
Epoch 20, Loss: 18.300419807434082
Epoch 21, Loss: 17.75965536557711
Epoch 22, Loss: 15.992747893700233
Epoch 23, Loss: 13.992955354543833
Epoch 24, Loss: 15.014331689247719
Epoch 25, Loss: 14.862646909860464
Epoch 26, Loss: 13.18836127794706
Epoch 27, Loss: 12.372035796825703
Epoch 28, Loss: 12.088921620295597
Epoch 29, Loss: 12.672788363236647
Epoch 30, 

16.551978626905996

In [39]:
model_clr.eval()
with torch.no_grad():
    reconstructed_data = []
    latent_z1 = []
    latent_z2 = []
    for x, _ in data_loader:
        recon_x, mu_z1, logvar_z1, mu_z2, logvar_z2 = model_clr(x, _)
        latent_z1.append(mu_z1)
        latent_z2.append(mu_z2)
        reconstructed_data.append(recon_x)
        
    reconstructed_data = torch.cat(reconstructed_data).numpy()
    latent_z1 = torch.cat(latent_z1).numpy()
    latent_z2 = torch.cat(latent_z2).numpy()

In [41]:
# Test with Z1
X_train_z1, X_test_z1, y_train_z1, y_test_z1 = train_test_split(latent_z1, Y_labels, test_size=0.3, random_state=42)
classifier_z1 = LogisticRegression(max_iter=1000)
classifier_z1.fit(X_train_z1, y_train_z1)

y_pred_z1 = classifier_z1.predict(X_test_z1)

print("Z1 Classifier Results:")
print(confusion_matrix(y_test_z1, y_pred_z1))
print(classification_report(y_test_z1, y_pred_z1))

# Test with Z2
X_train_z2, X_test_z2, y_train_z2, y_test_z2 = train_test_split(latent_z2, Y_labels, test_size=0.3, random_state=42)
classifier_z2 = LogisticRegression(max_iter=1000)
classifier_z2.fit(X_train_z2, y_train_z2)

y_pred_z2 = classifier_z2.predict(X_test_z2)

print("Z2 Classifier Results:")
print(confusion_matrix(y_test_z2, y_pred_z2))
print(classification_report(y_test_z2, y_pred_z2))

Z1 Classifier Results:
[[  0  87]
 [  0 157]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.64      1.00      0.78       157

    accuracy                           0.64       244
   macro avg       0.32      0.50      0.39       244
weighted avg       0.41      0.64      0.50       244

Z2 Classifier Results:
[[  0  87]
 [  0 157]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.64      1.00      0.78       157

    accuracy                           0.64       244
   macro avg       0.32      0.50      0.39       244
weighted avg       0.41      0.64      0.50       244



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa

In [43]:
#ILR attempt
X_tensor = torch.tensor(ilr_transformed_values, dtype=torch.float32) 
Y_tensor = torch.tensor(Y_labels, dtype=torch.float32)             
dataset = TensorDataset(X_tensor, Y_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [576]:
study = optuna.create_study(direction="minimize")  # Minimize the validation loss
study.optimize(objective, n_trials=100)

[I 2025-03-09 16:02:18,293] A new study created in memory with name: no-name-412fa711-90f5-4c50-8142-2345653b0cd5
/var/folders/02/jj1jlsn97sj550kl_82_8lp40000gn/T/ipykernel_11945/917332750.py:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



Epoch 1, Loss: 13970.083186222957
Epoch 2, Loss: 8178.947988656851
Epoch 3, Loss: 5939.270362267127
Epoch 4, Loss: 4630.290569598858
Epoch 5, Loss: 3836.404461200421
Epoch 6, Loss: 3316.2481595552886
Epoch 7, Loss: 2948.6704993614785
Epoch 8, Loss: 2677.603233924279
Epoch 9, Loss: 2475.8347825270434
Epoch 10, Loss: 2323.8569617638223
Epoch 11, Loss: 2208.426783635066
Epoch 12, Loss: 2119.4126328688403
Epoch 13, Loss: 2050.0308462289663
Epoch 14, Loss: 1995.1764690692607
Epoch 15, Loss: 1951.701387845553
Epoch 16, Loss: 1915.8888901930588
Epoch 17, Loss: 1886.996844951923
Epoch 18, Loss: 1863.264120248648
Epoch 19, Loss: 1843.5788034292368
Epoch 20, Loss: 1826.4140014648438
Epoch 21, Loss: 1812.233602670523
Epoch 22, Loss: 1800.1751685509314
Epoch 23, Loss: 1789.217533991887
Epoch 24, Loss: 1779.7351801945613
Epoch 25, Loss: 1770.99611722506
Epoch 26, Loss: 1763.3120563213643
Epoch 27, Loss: 1756.0624436598557
Epoch 28, Loss: 1749.70557814378
Epoch 29, Loss: 1743.64744919997
Epoch 30, L

[I 2025-03-09 16:02:19,573] Trial 0 finished with value: 2412.334492218801 and parameters: {'latent_dim_z1': 37, 'latent_dim_z2': 28, 'hidden_dim': 258, 'epochs': 51, 'causal_reg': 0.35234997088192144, 'learning_rate': 1.0296234334277778e-05}. Best is trial 0 with value: 2412.334492218801.


Epoch 45, Loss: 1665.829355093149
Epoch 46, Loss: 1660.792748084435
Epoch 47, Loss: 1656.4595149113582
Epoch 48, Loss: 1651.1680391751802
Epoch 49, Loss: 1645.845196063702
Epoch 50, Loss: 1640.4232928936299
Epoch 51, Loss: 1635.3319467397837
Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
Epoch 11, Loss: nan
Epoch 12, Loss: nan
Epoch 13, Loss: nan
Epoch 14, Loss: nan
Epoch 15, Loss: nan
Epoch 16, Loss: nan
Epoch 17, Loss: nan
Epoch 18, Loss: nan
Epoch 19, Loss: nan
Epoch 20, Loss: nan
Epoch 21, Loss: nan
Epoch 22, Loss: nan
Epoch 23, Loss: nan
Epoch 24, Loss: nan
Epoch 25, Loss: nan
Epoch 26, Loss: nan
Epoch 27, Loss: nan
Epoch 28, Loss: nan
Epoch 29, Loss: nan
Epoch 30, Loss: nan
Epoch 31, Loss: nan
Epoch 32, Loss: nan
Epoch 33, Loss: nan
Epoch 34, Loss: nan
Epoch 35, Loss: nan
Epoch 36, Loss: nan
Epoch 37, Loss: nan
Epoch 38, Loss: nan
Epoch 3

[W 2025-03-09 16:02:21,624] Trial 1 failed with parameters: {'latent_dim_z1': 32, 'latent_dim_z2': 21, 'hidden_dim': 103, 'epochs': 102, 'causal_reg': 0.05645624940105878, 'learning_rate': 0.0775995672478991} because of the following error: The value nan is not acceptable.
[W 2025-03-09 16:02:21,624] Trial 1 failed with value nan.


...

Epoch 191, Loss: nan
Epoch 192, Loss: nan
Epoch 193, Loss: nan
Epoch 194, Loss: nan
Epoch 195, Loss: nan
Epoch 196, Loss: nan
Epoch 197, Loss: nan
Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
Epoch 11, Loss: nan
Epoch 12, Loss: nan
Epoch 13, Loss: nan
Epoch 14, Loss: nan
Epoch 15, Loss: nan
Epoch 16, Loss: nan
Epoch 17, Loss: nan
Epoch 18, Loss: nan
Epoch 19, Loss: nan
Epoch 20, Loss: nan
Epoch 21, Loss: nan
Epoch 22, Loss: nan
Epoch 23, Loss: nan
Epoch 24, Loss: nan
Epoch 25, Loss: nan
Epoch 26, Loss: nan
Epoch 27, Loss: nan
Epoch 28, Loss: nan
Epoch 29, Loss: nan
Epoch 30, Loss: nan
Epoch 31, Loss: nan
Epoch 32, Loss: nan
Epoch 33, Loss: nan
Epoch 34, Loss: nan
Epoch 35, Loss: nan
Epoch 36, Loss: nan
Epoch 37, Loss: nan
Epoch 38, Loss: nan
Epoch 39, Loss: nan
Epoch 40, Loss: nan
Epoch 41, Loss: nan
Epoch 42, Loss: nan
Epoch 43, Loss: nan
Ep

[W 2025-03-09 16:08:29,742] Trial 98 failed with parameters: {'latent_dim_z1': 22, 'latent_dim_z2': 52, 'hidden_dim': 153, 'epochs': 200, 'causal_reg': 0.7257152234502848, 'learning_rate': 0.07053262497829192} because of the following error: The value nan is not acceptable.
[W 2025-03-09 16:08:29,742] Trial 98 failed with value nan.


Epoch 198, Loss: nan
Epoch 199, Loss: nan
Epoch 200, Loss: nan
Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
Epoch 11, Loss: nan
Epoch 12, Loss: nan
Epoch 13, Loss: nan
Epoch 14, Loss: nan
Epoch 15, Loss: nan
Epoch 16, Loss: nan
Epoch 17, Loss: nan
Epoch 18, Loss: nan
Epoch 19, Loss: nan
Epoch 20, Loss: nan
Epoch 21, Loss: nan
Epoch 22, Loss: nan
Epoch 23, Loss: nan
Epoch 24, Loss: nan
Epoch 25, Loss: nan
Epoch 26, Loss: nan
Epoch 27, Loss: nan
Epoch 28, Loss: nan
Epoch 29, Loss: nan
Epoch 30, Loss: nan
Epoch 31, Loss: nan
Epoch 32, Loss: nan
Epoch 33, Loss: nan
Epoch 34, Loss: nan
Epoch 35, Loss: nan
Epoch 36, Loss: nan
Epoch 37, Loss: nan
Epoch 38, Loss: nan
Epoch 39, Loss: nan
Epoch 40, Loss: nan
Epoch 41, Loss: nan
Epoch 42, Loss: nan
Epoch 43, Loss: nan
Epoch 44, Loss: nan
Epoch 45, Loss: nan
Epoch 46, Loss: nan
Epoch 47, Loss: nan
Epoch 

[W 2025-03-09 16:08:34,032] Trial 99 failed with parameters: {'latent_dim_z1': 21, 'latent_dim_z2': 52, 'hidden_dim': 146, 'epochs': 189, 'causal_reg': 0.7391718430918132, 'learning_rate': 0.07845032000076091} because of the following error: The value nan is not acceptable.
[W 2025-03-09 16:08:34,032] Trial 99 failed with value nan.


Epoch 186, Loss: nan
Epoch 187, Loss: nan
Epoch 188, Loss: nan
Epoch 189, Loss: nan
Best hyperparameters:  {'latent_dim_z1': 10, 'latent_dim_z2': 59, 'hidden_dim': 125, 'epochs': 190, 'causal_reg': 0.7380689811369623, 'learning_rate': 0.00014949572012646852}
Best validation loss:  362.6048607026998


In [578]:
print("Best hyperparameters: ", study.best_params)
print("Best validation loss: ", study.best_value)

Best hyperparameters:  {'latent_dim_z1': 10, 'latent_dim_z2': 59, 'hidden_dim': 125, 'epochs': 190, 'causal_reg': 0.7380689811369623, 'learning_rate': 0.00014949572012646852}
Best validation loss:  362.6048607026998


In [47]:
best_latent_dim_z1 = 10
best_latent_dim_z2 = 59
best_hidden_dim = 125
best_epochs = 190
best_learning_rate = 0.00014949572012646852
best_causal_reg = 0.7380689811369623

model_ilr = ModifiedVAE(614, best_latent_dim_z1,best_latent_dim_z2,best_hidden_dim)
optimizer = torch.optim.Adam(model_ilr.parameters(), lr=best_learning_rate)

train(model_ilr, data_loader, optimizer, best_epochs, best_causal_reg)

Epoch 1, Loss: 7277.553905780499
Epoch 2, Loss: 962.6521712083083
Epoch 3, Loss: 708.9931910588191
Epoch 4, Loss: 680.9330808199369
Epoch 5, Loss: 669.0604294996995
Epoch 6, Loss: 661.0105860783503
Epoch 7, Loss: 655.287107027494
Epoch 8, Loss: 650.4564878023588
Epoch 9, Loss: 646.2296330378606
Epoch 10, Loss: 642.2817112849309
Epoch 11, Loss: 638.9939704308143
Epoch 12, Loss: 635.9552905742938
Epoch 13, Loss: 632.710452740009
Epoch 14, Loss: 629.9885594294622
Epoch 15, Loss: 627.4507188063401
Epoch 16, Loss: 623.4884713979868
Epoch 17, Loss: 619.7812981238732
Epoch 18, Loss: 614.8439483642578
Epoch 19, Loss: 607.0066422682542
Epoch 20, Loss: 594.9738699106069
Epoch 21, Loss: 576.6617631178635
Epoch 22, Loss: 552.0675858717698
Epoch 23, Loss: 526.1634404109075
Epoch 24, Loss: 500.874514066256
Epoch 25, Loss: 476.44667287973255
Epoch 26, Loss: 456.90846487192005
Epoch 27, Loss: 440.9939751258263
Epoch 28, Loss: 427.8338177020733
Epoch 29, Loss: 418.0857708270733
Epoch 30, Loss: 407.4863

394.9844367980957

In [49]:
model_ilr.eval()
with torch.no_grad():
    reconstructed_data = []
    latent_z1 = []
    latent_z2 = []
    for x, _ in data_loader:
        recon_x, mu_z1, logvar_z1, mu_z2, logvar_z2 = model_ilr(x, _)
        latent_z1.append(mu_z1)
        latent_z2.append(mu_z2)
        reconstructed_data.append(recon_x)
        
    reconstructed_data = torch.cat(reconstructed_data).numpy()
    latent_z1 = torch.cat(latent_z1).numpy()
    latent_z2 = torch.cat(latent_z2).numpy()

In [51]:
# Test with Z1
X_train_z1, X_test_z1, y_train_z1, y_test_z1 = train_test_split(latent_z1, Y_labels, test_size=0.3, random_state=42)
classifier_z1 = LogisticRegression(max_iter=1000)
classifier_z1.fit(X_train_z1, y_train_z1)

y_pred_z1 = classifier_z1.predict(X_test_z1)

print("Z1 Classifier Results:")
print(confusion_matrix(y_test_z1, y_pred_z1))
print(classification_report(y_test_z1, y_pred_z1))

# Test with Z2
X_train_z2, X_test_z2, y_train_z2, y_test_z2 = train_test_split(latent_z2, Y_labels, test_size=0.3, random_state=42)
classifier_z2 = LogisticRegression(max_iter=1000)
classifier_z2.fit(X_train_z2, y_train_z2)

y_pred_z2 = classifier_z2.predict(X_test_z2)

print("Z2 Classifier Results:")
print(confusion_matrix(y_test_z2, y_pred_z2))
print(classification_report(y_test_z2, y_pred_z2))

Z1 Classifier Results:
[[  0  87]
 [  0 157]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.64      1.00      0.78       157

    accuracy                           0.64       244
   macro avg       0.32      0.50      0.39       244
weighted avg       0.41      0.64      0.50       244

Z2 Classifier Results:
[[  0  87]
 [  0 157]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.64      1.00      0.78       157

    accuracy                           0.64       244
   macro avg       0.32      0.50      0.39       244
weighted avg       0.41      0.64      0.50       244



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa